In [29]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models
import time, copy

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True

print(torch.cuda.is_available())

True


In [32]:
num_epochs = 20
batch_size = 32
learning_rate = 1e-3

In [54]:
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2,
                           contrast=0.2,
                           saturation=0.2,
                           hue=0.05),
    transforms.RandomAffine(degrees=0,translate=(0.05,0.05),shear=5),
    transforms.GaussianBlur(kernel_size=3,sigma=(0.1,2.0)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

In [34]:
train_dataset = datasets.ImageFolder(root='datasetmerged/train', transform=transform_train)
test_dataset = datasets.ImageFolder(root='datasetmerged/test', transform=transform_test)

In [35]:
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=4, pin_memory=True
)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False,
    num_workers=4, pin_memory=True
)


In [36]:
classes = ['fake', 'real']

Without frozen weights

In [37]:
model = models.resnet18(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 2)
# model.to(device)

c:\Users\Aditya Bhagat\Desktop\ml\AIimagedetector\AIimageDetector\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Aditya Bhagat\Desktop\ml\AIimagedetector\AIimageDetector\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [38]:
for params in model.parameters():
    params.requires_grad=False

In [39]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [40]:
print(next(model.parameters()).device)

cuda:0


In [41]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler=optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.5,patience=3)

In [42]:
dataloaders = {'train': train_loader, 'val': test_loader}
dataset_sizes = {'train': len(train_loader.dataset), 'val': len(test_loader.dataset)}

In [22]:
#generic function
# def train_model(model, criterion, optimizer, scheduler, num_epochs):
#     since = time.time()
#     best_model_wts = copy.deepcopy(model.state_dict())
#     best_acc = 0.0

#     for epoch in range(num_epochs):
#         print(f'Epoch {epoch}/{num_epochs - 1}')
#         print('-' * 10)

#         # Each epoch has a training and validation phase
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 model.train()  # Set model to training mode
#             else:
#                 model.eval()   # Set model to evaluation mode

#             running_loss = 0.0
#             running_corrects = 0

#             # Iterate over data.
#             for inputs, labels in dataloaders[phase]:
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)

#                 # forward
#                 with torch.set_grad_enabled(phase == 'train'):
#                     outputs = model(inputs)
#                     _, preds = torch.max(outputs, 1)
#                     loss = criterion(outputs, labels)

#                     # backward + optimize only if in training phase
#                     if phase == 'train':
#                         optimizer.zero_grad()
#                         loss.backward()
#                         optimizer.step()

#                 # Statistics
#                 running_loss += loss.item() * inputs.size(0)
#                 running_corrects += torch.sum(preds == labels.data)

#             if phase == 'train':
#                 scheduler.step()

#             epoch_loss = running_loss / dataset_sizes[phase]
#             epoch_acc = running_corrects.double() / dataset_sizes[phase]

#             print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

#             # Deep copy the model
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

#         print()

#     time_elapsed = time.time() - since
#     print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
#     print(f'Best val Acc: {best_acc:.4f}')

#     # Load best model weights
#     model.load_state_dict(best_model_wts)
#     return model


In [43]:
# def train_model(model, criterion, optimizer, scheduler,
#                 dataloaders, dataset_sizes, device,
#                 num_epochs, stage='warmup'):

#     import time, copy
#     since = time.time()
#     best_wts = copy.deepcopy(model.state_dict())
#     best_acc = 0.0

#     # Stage setup: freeze or unfreeze
#     if stage == 'warmup':
#         for name, p in model.named_parameters():
#             p.requires_grad = ('fc' in name)
#     else:
#         for p in model.parameters():
#             p.requires_grad = True

#     for epoch in range(num_epochs):
#         print(f'Epoch {epoch}/{num_epochs-1}  Stage: {stage}')
#         print('-' * 10)

#         epoch_loss = {'train': 0.0, 'val': 0.0}
#         epoch_acc  = {'train': 0.0, 'val': 0.0}

#         for phase in ['train', 'val']:
#             model.train() if phase=='train' else model.eval()
#             loader = dataloaders[phase]

#             running_loss = 0.0
#             running_corrects = 0

#             for inputs, labels in loader:
#                 inputs, labels = inputs.to(device), labels.to(device)

#                 with torch.set_grad_enabled(phase=='train'):
#                     outputs = model(inputs)
#                     loss = criterion(outputs, labels)
#                     _, preds = torch.max(outputs, 1)

#                     if phase == 'train':
#                         optimizer.zero_grad()
#                         loss.backward()
#                         optimizer.step()

#                 running_loss += loss.item() * inputs.size(0)
#                 running_corrects += (preds == labels).sum().item()

#             epoch_loss[phase] = running_loss / dataset_sizes[phase]
#             epoch_acc[phase]  = running_corrects / dataset_sizes[phase]
#             print(f'{phase} Loss: {epoch_loss[phase]:.4f} Acc: {epoch_acc[phase]:.4f}')

#         # Step scheduler ONCE per epoch
#         if stage == 'warmup':
#             scheduler.step(epoch_loss['train'])
#         else:
#             scheduler.step(epoch_loss['val'])

#         # Save best weights if current val acc is better
#         if epoch_acc['val'] > best_acc:  # Removed stage condition
#             best_acc = epoch_acc['val']
#             best_wts = copy.deepcopy(model.state_dict())

#         print()

#     time_elapsed = time.time() - since
#     print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
#     print(f'Best val Acc: {best_acc:.4f}')

#     model.load_state_dict(best_wts)
#     return model
def train_model(model, criterion, optimizer, scheduler,
                dataloaders, dataset_sizes, device,
                num_epochs, stage='warmup',
                patience=2, min_delta=1e-4):
    import time, copy
    since = time.time()
    best_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Early‐stopping trackers
    epochs_no_improve = 0
    best_val_loss = float('inf')

    # Stage setup: freeze or unfreeze
    if stage == 'warmup':
        for name, p in model.named_parameters():
            p.requires_grad = ('fc' in name)
    else:
        for p in model.parameters():
            p.requires_grad = True

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}  Stage: {stage}')
        print('-' * 10)

        epoch_loss = {'train': 0.0, 'val': 0.0}
        epoch_acc  = {'train': 0.0, 'val': 0.0}

        for phase in ['train', 'val']:
            model.train() if phase=='train' else model.eval()
            loader = dataloaders[phase]

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in loader:
                inputs, labels = inputs.to(device), labels.to(device)

                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += (preds == labels).sum().item()

            epoch_loss[phase] = running_loss / dataset_sizes[phase]
            epoch_acc[phase]  = running_corrects / dataset_sizes[phase]
            print(f'{phase} Loss: {epoch_loss[phase]:.4f} Acc: {epoch_acc[phase]:.4f}')

        # Scheduler step
        monitor_loss = epoch_loss['train'] if stage=='warmup' else epoch_loss['val']
        scheduler.step(monitor_loss)

        # Early‐stopping check (on validation loss)
        val_loss = epoch_loss['val']
        if best_val_loss - val_loss > min_delta:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            print(f'No improvement in val loss for {epochs_no_improve} epoch(s).')

        if epochs_no_improve >= patience:
            print(f'Early stopping triggered after {epoch+1} epochs.')
            break

        # Save best weights based on val acc
        if epoch_acc['val'] > best_acc:
            best_acc = epoch_acc['val']
            best_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_wts)
    return model


In [17]:
model = train_model(model, criterion, optimizer, scheduler,
                    dataloaders, dataset_sizes, device,
                    num_epochs=5, stage='warmup')

Epoch 0/4  Stage: warmup
----------
train Loss: 0.4480 Acc: 0.7906
val Loss: 0.5366 Acc: 0.7525

Epoch 1/4  Stage: warmup
----------
train Loss: 0.4316 Acc: 0.8027
val Loss: 0.5228 Acc: 0.7582

Epoch 2/4  Stage: warmup
----------
train Loss: 0.4278 Acc: 0.8030
val Loss: 0.5391 Acc: 0.7507
No improvement in val loss for 1 epoch(s).

Epoch 3/4  Stage: warmup
----------
train Loss: 0.4234 Acc: 0.8063
val Loss: 0.5699 Acc: 0.7438
No improvement in val loss for 2 epoch(s).
Early stopping triggered after 4 epochs.
Training complete in 14m 31s
Best val Acc: 0.7582


In [18]:
torch.save(model.state_dict(), 'resnet18_model_warmup4.pth')


In [19]:
optimizer = torch.optim.Adam([
    {'params': model.layer4.parameters(), 'lr': 1e-4},
    {'params': model.fc.parameters(),    'lr': 1e-3},
    {'params': [p for n,p in model.named_parameters() if 'layer4' not in n and 'fc' not in n],
     'lr': 1e-5}
])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=4)
model = train_model(model, criterion, optimizer, scheduler,
                    dataloaders, dataset_sizes, device,
                    num_epochs=15, stage='finetune')

Epoch 0/14  Stage: finetune
----------
train Loss: 0.2945 Acc: 0.8749
val Loss: 0.3496 Acc: 0.8617

Epoch 1/14  Stage: finetune
----------
train Loss: 0.2230 Acc: 0.9080
val Loss: 0.3188 Acc: 0.8814

Epoch 2/14  Stage: finetune
----------
train Loss: 0.1984 Acc: 0.9191
val Loss: 0.2449 Acc: 0.9055

Epoch 3/14  Stage: finetune
----------
train Loss: 0.1824 Acc: 0.9273
val Loss: 0.3642 Acc: 0.8973
No improvement in val loss for 1 epoch(s).

Epoch 4/14  Stage: finetune
----------
train Loss: 0.1666 Acc: 0.9337
val Loss: 0.2886 Acc: 0.9028
No improvement in val loss for 2 epoch(s).
Early stopping triggered after 5 epochs.
Training complete in 20m 15s
Best val Acc: 0.9055


In [20]:
torch.save(model.state_dict(), 'resnet18_model4.pth')

In [21]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [22]:
preprocess = transforms.Compose([
    transforms.Resize(224),          # Resize to match training input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Same as training
])

In [23]:
def predict_image(image_path):
    # Open and convert to RGB
    image = Image.open(image_path).convert('RGB')
    
    # Apply preprocessing
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)  # Add batch dimension
    input_batch = input_batch.to(device)     # Move to device

    return input_batch

In [24]:
def get_prediction(input_batch):
    with torch.no_grad():
        output = model(input_batch)
    
    # Convert to probabilities
    probabilities = F.softmax(output, dim=1)[0]
    predicted_class_idx = torch.argmax(probabilities).item()
    confidence = probabilities[predicted_class_idx].item()
    predicted_class = classes[predicted_class_idx]
    
    return predicted_class, confidence

In [ ]:
from PIL import Image
image_path = r"" 

# Load and preprocess
input_batch = predict_image(image_path)

# Get prediction
class_name, confidence = get_prediction(input_batch)
print(f'Predicted: {class_name} with {confidence*100:.2f}% confidence')

Predicted: fake with 100.00% confidence


In [ ]:
checkpoint = torch.load(r"", map_location='cuda')
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [21]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  